In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import requests
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from IPython.display import display

In [6]:
url = "https://www.madlan.co.il/for-sale/צפת-ישראל"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}  # This is chrome, you can set whatever browser you like
response = requests.get(url,headers= headers)
soup = BeautifulSoup(response.content, 'html.parser')
if response.status_code == 200:
    print("Success")
else:
    print("Failure")

Success


<!DOCTYPE html>
<style data-emotion-css="44lp2d">html,body,div,span,applet,object,iframe,h1,h2,h3,h4,h5,h6,p,blockquote,pre,a,abbr,acronym,address,big,cite,code,del,dfn,em,img,ins,kbd,q,s,samp,small,strike,strong,sub,sup,tt,var,b,u,i,center,dl,dt,dd,ol,ul,li,fieldset,form,label,legend,table,caption,tbody,tfoot,thead,tr,th,td,article,aside,canvas,details,embed,figure,figcaption,footer,header,hgroup,menu,nav,output,ruby,section,summary,time,mark,audio,video{margin:0;padding:0;border:0;font-size:100%;font:inherit;vertical-align:baseline;}article,aside,details,figcaption,figure,footer,header,hgroup,menu,nav,section{display:block;}body{line-height:1;overscroll-behavior:none;}ol,ul{list-style:none;}blockquote,q{quotes:none;}blockquote:before,blockquote:after,q:before,q:after{content:'';content:none;}table{border-collapse:collapse;border-spacing:0;}*{box-sizing:border-box;}body{font-family:novemberHebrew,Muli,sans-serif;}</style><style data-emotion-css="fyvm37">#overlay-root > div:not(:last-o

In [18]:
citys=re.findall('ב(.+)',soup.find('h1',{'class':'css-r8epb0 e10ue7076'}).get_text())[0]
assets=soup.find_all('div',{'class':'css-s2aw04 e1sx3tzs2'})
Street=[]
street_num=[]
area=[]
area_in_city=[]
room_num=[]
type_asset=[]
price=[]
for asset in assets:
    is_new_project=asset.find_all("div",{"class":"css-1i00gl6 e1sx3tzs17","data-auto":"property-class"})
    moda=asset.find_all("div",{"class":"css-1i00gl6 e1sx3tzs17","data-auto":"property-size"})
    if (is_new_project!=[] and moda!=[]):   
        #street
        for_street=asset.find_all("div",{"class":"css-1i00gl6 e1sx3tzs17"})[-1]
        if (re.findall(r'^(.*?)(?:\d+|,)',for_street.get_text()))==[]:
            Street.append(re.findall("(.+)\s*",for_street.get_text())[0])
        else:
            Street.append(re.findall(r'^(.*?)(?:\d+|,)',for_street.get_text())[0])
        #street number 
        if (re.findall('([0-9]+).*',for_street.get_text()))==[]:
            street_num.append("nan")
        else:
            street_num.append(re.findall('([0-9]+).*',for_street.get_text())[0])
        # area in the city
        if (re.findall(",(.*)",for_street.get_text()))==[]:
            area_in_city.append("nan")
        else:
            area_in_city.append(re.findall(",(.*)",for_street.get_text())[0])
        #area of the hous
        for_area=asset.find_all("div",{"class":"css-1i00gl6 e1sx3tzs17","data-auto":"property-size"})
        if (for_area)==[]:
            area.append("nan")
        else:
            area.append(for_area[0].get_text())
        for_room=asset.find_all("div",{"class":"css-1i00gl6 e1sx3tzs17","data-auto":"property-rooms"})
        if for_room==[]:
            room_num.append("nan")
        else:
            try:
                room_num.append(re.findall("([0-9]+.[0-9]*).*",for_room[0].get_text())[0])
            except:
                room_num.append(re.findall(r"\bחדר\b\s*(\S+)",for_room[0].get_text())[0])
        #the a[artment type
        for_type=asset.find_all("div",{"class":"css-1i00gl6 e1sx3tzs17","data-auto":"property-class"})
        if (for_type)==[]:
            type_asset.append("nan")
        else:
            type_asset.append(for_type[0].get_text())
        #price
        for_price=asset.find_all("div",{"class":"css-19tpcfu eu6esji3","data-auto":"property-price"})
        if (for_price)==[]:
            price.append("nan")
        else:
            price.append(re.findall(r'[\d,]+',for_price[0].get_text())[0])
    else:
        continue

In [21]:
df={ 'City': citys,'type':type_asset, 'room_number': room_num,"Area":area,"Street":Street,"number_in_street":street_num,"city_area":area_in_city,"price":price}
df = pd.DataFrame(df)
df